# GPT를 활용해 리뷰요약 best prompt 작성
- 우리 서비스에서 사용할 GPT모델 : gpt3.5-turbo

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# 환경변수 로딩
load_dotenv('../.env', override=True)

# 메모리에 로딩된 값을 api_key 변수에 대입
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


# 데이터 전처리
- 리뷰 데이터 파일 로딩
- 리뷰 요약 기준(평점: 1~5)
    - good : 평점 5인것
    - bad : 평점이 4이하인것
- 각 리뷰마다 specital token
    - '[REVIEW_START]' + ['review'] + '[REVIEW_END]'
- 리뷰를 하나의 문자열로 통합
    - reviews_good, reviews_bad

In [ ]:
import datetime
current_date = datetime.datetime.now()
# 현재를 기준으로 6개월전
date_boundary = current_date - datetime.timedelta(days=6*30)

datetime.datetime(2024, 12, 6, 12, 34, 11, 330185)

- 하나의 숙박시설에 대한 리뷰데이터 전처리 함수 정의

In [18]:
import json
from dateutil import parser

# 정의부
def preprocess_reviews(data_path):
    # 1. 파일 데이터 로딩
    with open(data_path, 'r') as f:
        review_list = json.load(f)
    # print(review_list)
    # 2. good, bad 분리
    # - 리뷰 요약 기준(평점: 1~5)
    # - good : 평점 5인것
    # - bad : 평점이 4이하인것
    # - 각 리뷰마다 specital token
    #     - '[REVIEW_START]' + ['review'] + '[REVIEW_END]'
    # - 리뷰를 하나의 문자열로 통합
    #     - reviews_good, reviews_bad
    # good과 bad 리뷰를 저장하기 위해, 변수 초기화
    # 3. 현재 이전 6개월 데이터 사용을 위해 날짜계산
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)
    # print(date_boundary)
    reviews_good, reviews_bad = [], []
    for r in review_list:
        review_date_str = r['date']
        try:
            # 리뷰 날짜 문자열을 datetime 객체로 변환
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            # 정상 날짜 데이터 아닌경우 현재 날짜 대입
            review_date = current_date

        # 날짜 바운더리 보다 작으면, pass
        if review_date < date_boundary:
            continue

        if r['stars'] == 5:
            # specital token 추가
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            # specital token 추가
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    # 3. good, bad 리뷰 각각 문자열로 만들기.
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)
    # print(reviews_good_text)
    # print(reviews_bad_text)
    return reviews_good_text, reviews_bad_text  

In [19]:
# 호출부  
data_path = "res/1.gangneung_mare.json"
reviews_good, reviews_bad = preprocess_reviews(data_path)

# best 프롬프트를 만들기위해 응답결과 평가하기